In [17]:
import torch, pickle, math, os
import numpy as np
from os.path import join, dirname, expanduser, exists
from torch.nn.functional import softmax
from collections import defaultdict
lab_dnn = torch.load("lab_dnn.pt")
out = torch.load("out.pt")
lab_conc = np.load("lab_conc.npy", allow_pickle=True)
mapping = np.load("mapping.npy")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

conf_dir = join(expanduser("~"), "UPM", "conf")
phones_filepath = join(conf_dir, "articulatory_features", "phone_attributes_filtered.txt")
feats_filepath = join(conf_dir, "articulatory_features", "feature_vectors.txt")
extensions_filepath = join(conf_dir, "articulatory_features", "extensions_filtered.txt")
curr_dir = os.getcwd()

In [3]:
arr = np.arange(0,1,0.1)

In [2]:
def get_phones(filename, confdir):
    filepath = join(confdir, filename)
    
    with open(filepath, "r", encoding="utf-8") as f:
        out = [line.split()[2] for line in f]
        
    out = [x for x in out if x.lower() not in ["sil", "unk"]]
    return out

In [4]:
def combine_all_phones(langs, confdir):
    all_phones = []
    conf_maps = join(confdir, "phone_maps")
    for lang in langs:
        filename = "{}_phone_map.txt".format(lang)
        lang_phones = get_phones(filename, conf_maps)
        all_phones += lang_phones
    all_phones = list(set(all_phones))
    return all_phones

langs = ["BG", "CR", "HA", "PL", "TU", "UA", "SA", "SW"]
all_phones = combine_all_phones(langs, conf_dir)


In [12]:
def get_diphthongs(all_phones):
    vowels = ["a", "e", "i", "o", "u", "y"]
    diphthongs = []
    for phone in all_phones:
        if len(phone) > 1:
            if phone[0] in vowels and phone[1] in vowels:
                diphthongs.append(phone)
    return diphthongs

dips = get_diphthongs(all_phones)

In [22]:
def get_diphthong_atts(dips, conf_dir):
    feat_file = join(conf_dir, "articulatory_features", "feature_vectors.txt")
    with open(feat_file, "r") as f:
        feat_lines = f.read().splitlines()
    header = feat_lines[0].split()
    
    dips_dict = defaultdict(list)
    for line in feat_lines:
        entry = line.split()
        phone = entry[0]
        if phone in dips:
            dip = entry[0]
            for idx, att in enumerate(entry):
                if att == "1":
                    dips_dict[dip].append(header[idx])
    return dips_dict
get_diphthong_atts(dips, conf_dir)

defaultdict(list,
            {'ai': ['close', 'front', 'open', 'unrounded', 'vowel'],
             'au': ['back',
              'close',
              'front',
              'open',
              'rounded',
              'unrounded',
              'vowel'],
             'ya': ['close', 'front', 'open', 'rounded', 'unrounded', 'vowel'],
             'yu': ['back', 'close', 'front', 'rounded', 'vowel']})

In [26]:
dips

['yu', 'ai', 'ya', 'au']

In [6]:
lab_chunks = np.load("lab_chunks.npy", allow_pickle=True)
fea_chunks = np.load("fea_chunks.npy", allow_pickle=True)

In [78]:
# TODO convert phone to phones.txt when reading

In [118]:
(lab_single == pred).sum().detach().cpu().numpy()

array(265, dtype=int64)

In [31]:
def filter_valid_extensions(extensions_filepath, phones_idx_dict, feat_idx_dict):
    vowel_extensions = []
    consonant_extensions = []
    with open(extensions_filepath, "r") as f:
        lines = f.read().splitlines()
        
    for line in lines:
        is_valid = False
        entry = line.split()
        extension = entry[0]
        # Check if extension occurs in any of the phones
        for phone in phones_idx_dict:
            if extension in phone:
                is_valid = True
        
        if is_valid:
            extension_atts = [feat_idx_dict[x] for x in entry[1:]]
            if extension in [":"]: # This extension seems to only apply to vowels
                vowel_extensions.append([extension] + extension_atts)
            else:
                consonant_extensions.append([extension] + extension_atts)
    return vowel_extensions, consonant_extensions

def get_phone_idx_dict(phones_txt_filepath):
    with open(phones_txt_filepath, "r") as f:
        phone_to_idx_dict = {}
        for line in f:
            entry = line.split()
            phone = entry[0]
            idx = int(entry[1])
            phone_to_idx_dict[phone] = idx
    return phone_to_idx_dict

def mapping_to_labels(mapping, lab_dnn):
    lab_detached = lab_dnn.detach().cpu().numpy()
    outputs = np.zeros(lab_detached.shape[0])
    for lab_idx, lab in enumerate(lab_detached):
        if sum(lab) == 0:
            outputs[lab_idx] = 1
        else:
            for mapping_idx, map_val in enumerate(mapping):
                if np.array_equal(map_val, lab):
                    outputs[lab_idx] = mapping_idx
    return outputs

def get_feat_idx(feats_filepath):
    with open(feats_filepath, "r") as f:
        header = f.readline()
    features = header.split()[1:]
    feat_idx_dict = {}
    for idx, feat in enumerate(features):
        feat_idx_dict[feat] = idx
    return feat_idx_dict
        
def add_extensions(extensions, phones, curr_phone, phone_to_idx_dict, remove_val, delete_vowel_consonant_idx):
    # Each extension is of the format [":", 3, 2]
    
    if curr_phone[0] not in ['yu', 'ai', 'ya', 'au']:
        for extension in extensions:
            # Only add new attributes
            atts_to_add = [x for x in extension[1:] if x not in curr_phone[1:]]
            if len(atts_to_add) > 0:
                ext_phone = curr_phone[0] + extension[0]
                if ext_phone in phone_to_idx_dict:
                    ext_phone_idx = phone_to_idx_dict[ext_phone]
                else:
                    ext_phone_idx = 0
                ext_feat = [ext_phone_idx] + curr_phone[1:] + atts_to_add
                if delete_vowel_consonant_idx:
                    ext_feat.remove(remove_val)
                phones.append(ext_feat)
    return phones
    
def get_phones(phones_filepath, feat_idx_dict, phone_to_idx_dict, ve, ce, include_extensions=True, delete_vowel_consonant_idx=True):
    # ve, ce are vowel and consonant extensions to add
    with open(phones_filepath, "r") as f:
        lines = f.read().splitlines()
    vowel_phones = []
    consonant_phones = []
    vowel_idx = feat_idx_dict["vowel"]
    consonant_idx = feat_idx_dict["consonant"]  
    
    for line in lines:
        # Entry = ["a", "front", "close", etc.]
        entry = line.split()
        is_vowel = False
        # Convert to indices for speed
        for idx, att in enumerate(entry):
            if att == "vowel":
                remove_idx = idx
                is_vowel = True
            elif att == "consonant":
                remove_idx = idx
                
            if att in feat_idx_dict:
                entry[idx] = feat_idx_dict[att]
                
        if is_vowel and include_extensions:
            vowel_phones = add_extensions(ve, vowel_phones, entry, phone_to_idx_dict, vowel_idx, delete_vowel_consonant_idx)
        elif include_extensions:
            consonant_phones = add_extensions(ce, consonant_phones, entry, phone_to_idx_dict, consonant_idx, delete_vowel_consonant_idx)
 
        # Entry[0] is the phone e.g. "a"
        phone = entry[0]
        # Special case for J\: (rare incidence of consonant + long in Turkish)
        if phone == "J\\" and "J\\:" in phone_to_idx_dict:
            long_idx = feat_idx_dict["long"]
            extra_phone_idx = phone_to_idx_dict["J\\:"]
            extra_feat = [extra_phone_idx] + entry[1:] + [long_idx]
            extra_feat.remove(consonant_idx)
            consonant_phones.append(extra_feat)
            
        if phone in phone_to_idx_dict:
            entry[0] = phone_to_idx_dict[phone]
        else:
            # Set unknown phones to <eps> ?
            entry[0] = 0
        if delete_vowel_consonant_idx:
            del entry[remove_idx]
        if is_vowel:
            vowel_phones.append(entry)
        else:
            consonant_phones.append(entry)

    return vowel_phones, consonant_phones

# Need to initially split between vowels and consonants before further on features
def initial_split(phones, feat_idx_dict):
    vowel_phones = []
    consonant_phones = []
    vowel_idx = feat_idx_dict["vowel"]
    consonant_idx = feat_idx_dict["consonant"]
    for phone in phones:
        if vowel_idx in phone:
            phone.remove(vowel_idx)
            vowel_phones.append(np.asarray(phone))

        elif consonant_idx in phone:
            phone.remove(consonant_idx)
            consonant_phones.append(np.asarray(phone))
                    
    vowel_phones = np.asarray(vowel_phones)
    consonant_phones = np.asarray(consonant_phones)
    return vowel_phones, consonant_phones

def save_variables(feat_idx_dict, vp, cp, vp_idx, cp_idx, filename=None, save_dir=None):
    if save_dir is None:
        save_dir = os.getcwd()
    if filename is None:
        save_path = join(save_dir, "feat_vars.pkl")
    else:
        save_path = join(save_dir, filename + ".pkl")
    with open(save_path, 'wb') as f:
        pickle.dump([feat_idx_dict, vp, cp, vp_idx, cp_idx], f)

feats_filepath = join(conf_dir, "articulatory_features", "feature_vectors.txt")
phones_filepath = join(conf_dir, "articulatory_features", "phone_attributes_filtered.txt")
phones_list_filepath = join(dirname(os.getcwd()), "notes", "phones.txt")
phone_idx_dict = get_phone_idx_dict(phones_list_filepath)
feat_idx_dict = get_feat_idx(feats_filepath)
ve, ce = filter_valid_extensions(extensions_filepath, phone_idx_dict, feat_idx_dict)


vp, cp = get_phones(phones_filepath, feat_idx_dict, phone_idx_dict, ve, ce)

# Get tensor of consonants e.g [0 3 4 5 112 1] - avoids excessive converting to and from tensors
cp_idx = [x[0] for x in cp]
cp_idx = torch.from_numpy(np.asarray(cp_idx))

# Get tensor of consonants e.g [0 3 4 5 112 1] - avoids excessive converting to and from tensors
vp_idx = [x[0] for x in vp]
vp_idx = torch.from_numpy(np.asarray(vp_idx))

save_variables(feat_idx_dict, vp, cp, vp_idx, cp_idx)

In [32]:
vp

[[35, 34, 21, 45, 29],
 [34, 34, 21, 45],
 [53, 12, 21, 45, 29],
 [52, 12, 21, 45],
 [64, 11, 21, 45, 29],
 [63, 11, 21, 45],
 [81, 12, 6, 42, 29],
 [80, 12, 6, 42],
 [101, 11, 6, 42, 29],
 [100, 11, 6, 42],
 [111, 11, 21, 42, 29],
 [110, 11, 21, 42],
 [12, 34, 6, 45, 29],
 [11, 34, 6, 45],
 [16, 35, 21, 45, 29],
 [15, 35, 21, 45],
 [0, 31, 21, 45, 29],
 [19, 31, 21, 45],
 [0, 31, 9, 45, 29],
 [0, 31, 9, 45],
 [0, 11, 6, 45, 29],
 [24, 11, 6, 45],
 [0, 35, 6, 42, 29],
 [26, 35, 6, 42],
 [0, 34, 6, 42, 29],
 [0, 34, 6, 42],
 [0, 31, 6, 42, 29],
 [31, 31, 6, 42],
 [0, 31, 9, 42, 29],
 [0, 31, 9, 42],
 [0, 35, 6, 45, 29],
 [0, 35, 6, 45],
 [0, 31, 21, 42, 29],
 [0, 31, 21, 42],
 [0, 12, 35, 9, 42, 45, 29],
 [10, 12, 35, 9, 42, 45],
 [118, 32, 21, 45, 29],
 [117, 32, 21, 45],
 [119, 11, 9, 42, 29],
 [0, 11, 9, 42],
 [0, 11, 9, 45, 29],
 [3, 11, 9, 45],
 [0, 12, 21, 42, 29],
 [4, 12, 21, 42],
 [0, 35, 9, 45, 29],
 [0, 35, 9, 45],
 [0, 35, 9, 42, 29],
 [0, 35, 9, 42],
 [0, 32, 9, 29],
 [0, 3

In [34]:
def build_universal_phonemap(phones_filepath, ve, ce, write_filepath=None):
    with open(phones_filepath, "r") as f:
        lines = f.read().splitlines()
    
    write_lines = ["<eps> 0", "sil 1"]
    next_num = 2
    
    for line in lines:
        curr_phone = line.split()
        phone = curr_phone[0]
        write_lines.append("{} {}".format(phone, next_num))
        next_num += 1
        if "vowel" in curr_phone:
            if phone not in ['yu', 'ai', 'ya', 'au']:
                for ext_list in ve:
                    ext = ext_list[0]
                    phone_ext = phone + ext
                    write_lines.append("{} {}".format(phone_ext, next_num))
                    next_num += 1
                
        elif "consonant" in curr_phone:
            for ext_list in ce:
                ext = ext_list[0]
                phone_ext = phone + ext
                write_lines.append("{} {}".format(phone_ext, next_num))
                next_num += 1
                
    # Special case of "long" applying to consonant
    write_lines.append("J\\: {}".format(next_num))
                
    if write_filepath is None:
        write_filepath = join(os.getcwd(), "universal_phones.txt")
        
    with open(write_filepath, "w") as f:
        print("Writing to {}".format(write_filepath))
        for line in write_lines:
            f.write(line + "\n")
    return write_filepath
            
build_universal_phonemap(phones_filepath, ve, ce)

Writing to C:\Users\Paul\UPM\pytorch-kaldi\universal_phones.txt


'C:\\Users\\Paul\\UPM\\pytorch-kaldi\\universal_phones.txt'

In [7]:
# For universal phone stuff
feats_filepath = join(conf_dir, "articulatory_features", "feature_vectors.txt")
phones_list_filepath = join(os.getcwd(), "universal_phones.txt")
phones_filepath = join(conf_dir, "articulatory_features", "phone_attributes_filtered.txt")
phone_idx_dict = get_phone_idx_dict(phones_list_filepath)
feat_idx_dict = get_feat_idx(feats_filepath)
ve, ce = filter_valid_extensions(extensions_filepath, phone_idx_dict, feat_idx_dict)


vp, cp = get_phones(phones_filepath, feat_idx_dict, phone_idx_dict, ve, ce)

# Get tensor of consonants e.g [0 3 4 5 112 1] - avoids excessive converting to and from tensors
cp_idx = [x[0] for x in cp]
cp_idx = torch.from_numpy(np.asarray(cp_idx))

# Get tensor of consonants e.g [0 3 4 5 112 1] - avoids excessive converting to and from tensors
vp_idx = [x[0] for x in vp]
vp_idx = torch.from_numpy(np.asarray(vp_idx))

In [188]:
vp_ne, cp_ne = get_phones(phones_filepath, feat_idx_dict, phone_idx_dict, ve, ce, include_extensions=False)

cp_idx_ne = [x[0] for x in cp_ne]
cp_idx_ne = torch.from_numpy(np.asarray(cp_idx_ne))

# Get tensor of consonants e.g [0 3 4 5 112 1] - avoids excessive converting to and from tensors
vp_idx_ne = [x[0] for x in vp_ne]
vp_idx_ne = torch.from_numpy(np.asarray(vp_idx_ne))

save_variables(feat_idx_dict, vp_ne, cp_ne, vp_idx_ne, cp_idx_ne, filename="feat_vars_ne")

In [16]:
def load_variables(filename=None, load_dir=None):
    if load_dir is None:
        load_dir = os.getcwd()
    if filename is None:
        load_path = join(load_dir, "feat_vars.pkl")
    else:
        load_path = join(load_dir, filename + ".pkl")
    with open(load_path, 'rb') as f:
        feat_idx_dict, vp, cp, vp_idx, cp_idx = pickle.load(f)
    return feat_idx_dict, vp, cp, vp_idx, cp_idx

feat_idx_dict, vp, cp, vp_idx, cp_idx = load_variables()

In [8]:
def convert_to_pred(out, feat_idx_dict, vp, cp, vp_idx, cp_idx):
    # split into silence, vowels and consonants
    vowel_idx = feat_idx_dict["vowel"]
    consonant_idx = feat_idx_dict["consonant"]
    vowels = out[:, vowel_idx]
    consonants = out[:, consonant_idx]
    sums = vowels + consonants
    # There is roughly a 50% chance this phone is non-silence.
    sum_bools = sums > 0.5
    # Take max of vowel vs. consonant
    vowel_bools = vowels > consonants
    consonant_bools = ~vowel_bools
    
    vowel_idx = sum_bools & vowel_bools 
    consonant_idx = sum_bools & consonant_bools
    silence_idx = ~sum_bools

    vowels = out[vowel_idx]
    consonants = out[consonant_idx]

    # take the slice from [1:] (all the atts idx)
    # sum the value of these attributes. 
    # divide by the number of attributes to normalise
    v_scores = [vowels[:,slice_idx[1:]].sum(dim=1)/(len(slice_idx)-1) for slice_idx in vp]
    # Stack to shape [29, N], where each 29 is the score for each vowel
    v_scores = torch.stack(v_scores, dim=0)
    # Get max phone idx, and convert this to the corresponding phone
    v_out = vp_idx[v_scores.argmax(dim=0)] 

    c_scores = [consonants[:,slice_idx[1:]].sum(dim=1)/(len(slice_idx)-1) for slice_idx in cp]
    # Stack to shape [86, N] where each 86 is the score for each consonant
    c_scores = torch.stack(c_scores, dim=0)
    c_out = cp_idx[c_scores.argmax(dim=0)]
    
    # Fill final output
    final_out = torch.zeros(out.size()[0]).cuda()
    final_out.masked_scatter_(consonant_idx, c_out.cuda().float())
    final_out.masked_scatter_(vowel_idx, v_out.cuda().float())
    final_out.masked_fill_(silence_idx, 1)
    final_out = final_out.int()
    
    return final_out

pred_2 = convert_to_pred(out, feat_idx_dict, vp, cp, vp_idx, cp_idx)

In [9]:
pred_2

tensor([178,   1,   1, 196, 178, 196,   2, 196, 196,   1,   1, 196,   2, 196,
          2, 196, 196,   1,   1,  60,   2, 196,   2,  16, 196,   1,   1,  60,
          2,   2,   2,  16, 196,   1,   1,  60,   2,   2, 248, 196, 196,   1,
          1,  60,   2,   2, 248, 196, 196,   1,   1,  60,   2,   2, 248, 196,
        196,   1,   1,  60,   2,   2, 248, 196, 196,   1,   1,  16,   2,   2,
        248,   2, 196,   1,   1,  16,   2,   2, 248,   2, 196,   1,   1,   2,
          2,   2, 248,   2,  16,   1,   1,   2,   2,   2, 248,   2,  16,   1,
          1,  60,   2,   2, 248,   2,  16,   1,   1,  16,   2,   2, 248,   2,
         16,   1,   1,  16,   2,   2, 248,   2,  16,   1,   1,  16,   2,   2,
        248,   2,  16,   1,   1,  16,   2,   2, 248, 110, 196,   1,   1,  16,
          2,   2,   2, 110, 196,   1,   1,   2,   2, 110,   2, 110, 196,   1,
          1,   2,   2, 110, 248, 196, 196,   1,   1,   2,   2, 110, 110, 196,
        196,   1,   1,   2,   2,  16, 248, 178, 196,   1,  16,  

In [6]:
pred

tensor([ 89,   1,   1,  92,  89,  92,  34,  92,  92,   1,   1,  92,  34,  92,
         34,  92,  92,   1,   1,  63,  34,  92,  34,  43,  92,   1,   1,  63,
         34,  34,  34,  43,  92,   1,   1,  63,  34,  34, 114,  92,  92,   1,
          1,  63,  34,  34, 114,  92,  92,   1,   1,  63,  34,  34, 114,  92,
         92,   1,   1,  63,  34,  34, 114,  92,  92,   1,   1,  43,  34,  34,
        114,  34,  92,   1,   1,  43,  34,  34, 114,  34,  92,   1,   1,  34,
         34,  34, 114,  34,  43,   1,   1,  34,  34,  34, 114,  34,  43,   1,
          1,  63,  34,  34, 114,  34,  43,   1,   1,  43,  34,  34, 114,  34,
         43,   1,   1,  43,  34,  34, 114,  34,  43,   1,   1,  43,  34,  34,
        114,  34,  43,   1,   1,  43,  34,  34, 114,  77,  92,   1,   1,  43,
         34,  34,  34,  77,  92,   1,   1,  34,  34,  77,  34,  77,  92,   1,
          1,  34,  34,  77, 114,  92,  92,   1,   1,  34,  34,  77,  77,  92,
         92,   1,   1,  34,  34,  43, 114,  89,  92,   1,  43,  

In [45]:
# Takes max value for the zero (unknown) phone
def reduce_zeros(scores, idx):
    zero_idx = idx == 0
    zero_count = zero_idx.sum()
        
    zero = torch.zeros(1)
        
    # Only keep highest zero-score
    max_zero_score = scores[zero_idx].max(axis=0)[0]
    # Add 1-dim for concatenation
    max_zero_score.unsqueeze_(0)

    # filter out the original zero rows
    scores_out = torch.cat((max_zero_score, scores[~zero_idx,:]), 0).cuda()
    idx_out = torch.cat((zero, idx[~zero_idx].float())).cuda()
        
    # idx_out is now [0, x, y, ...]
    # scores_out is now [zero_scores, x_scores, y_scores, ...]
    # zero_scores is the mean zero (unknown) score
    return scores_out, idx_out

In [125]:
def convert_to_scores(out, feat_idx_dict, phone_idx_dict, vp, cp, vp_idx, cp_idx):
    # split into silence, vowels and consonants
    vowel_idx = feat_idx_dict["vowel"]
    consonant_idx = feat_idx_dict["consonant"]
    vowels = out[:, vowel_idx]
    consonants = out[:, consonant_idx]
    sums = vowels + consonants
    # There is roughly a 50% chance this phone is non-silence.
    sum_bools = sums > 0.5
    # Take max of vowel vs. consonant
    vowel_bools = vowels > consonants
    consonant_bools = ~vowel_bools
    
    vowel_idx = sum_bools & vowel_bools 
    consonant_idx = sum_bools & consonant_bools
    silence_idx = ~sum_bools

    vowels = out[vowel_idx]
    consonants = out[consonant_idx]
    
    n_phones = len(phone_idx_dict)
    
    # Output is N x #phones
    out_scores = torch.zeros((out.size()[0], n_phones))
    
    combined_v_scores = get_all_phone_scores(vowels, vp, vp_idx, n_phones)
    combined_c_scores = get_all_phone_scores(consonants, cp, cp_idx, n_phones)

    # Fill final output
    final_out = torch.zeros((out.size()[0], n_phones)).cuda()
    final_out[consonant_idx,:] =  combined_c_scores
    final_out[vowel_idx,:] = combined_v_scores
    
    silence_sum = sums[silence_idx]  # Will be values < 0.5
    silence_prob = 1 - silence_sum # Will be values > 0.5
    non_silence_probs = silence_sum / (n_phones) # Distribute likelihood of non-silence across attributes 
    non_silence_probs.unsqueeze_(dim=1)
    final_out[silence_idx,:] =  non_silence_probs.repeat(1, n_phones)
    
    final_out[silence_idx,1] = torch.exp(1 - sums[silence_idx])  # Fill the silent indices with the probability they were silence
    final_out = final_out.cuda()
    
    final_out[final_out == 0] = -math.inf
    final_out = softmax(final_out, dim=1)
    return final_out

def get_all_phone_scores(out, phones, phone_idx, total_seen_phones):
    # take the slice from [1:] (all the atts idx)
    # sum the value of these attributes. 
    # divide by the number of attributes to normalise
    scores = [out[:,slice_idx[1:]].sum(dim=1)/(len(slice_idx)-1) for slice_idx in phones]
    # Stack to shape [# vowel phones, N]
    scores = torch.stack(scores, dim=0)
    n = scores.shape[1]
    
    scores, phone_idx = reduce_zeros(scores, phone_idx)
    
    combined_scores = torch.zeros((n, total_seen_phones))
    for idx, col_idx in enumerate(phone_idx.long()):
        combined_scores[:,col_idx] = scores[idx,:]
        
    return combined_scores.cuda()
        
final_out_2 = convert_to_scores(out, feat_idx_dict, phone_idx_dict, vp, cp, vp_idx, cp_idx)

tensor(0.0004, device='cuda:0', grad_fn=<SelectBackward>)


In [130]:
final_out_2.size()

torch.Size([800, 122])

In [131]:
arr = final_out_2.detach().cpu().numpy()

In [132]:
arr

array([[0.01377007, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00735575, 0.10995455, 0.00735575, ..., 0.00735575, 0.00735575,
        0.00735575],
       [0.00763683, 0.07594381, 0.00763683, ..., 0.00763683, 0.00763683,
        0.00763683],
       ...,
       [0.03710816, 0.        , 0.        , ..., 0.02822955, 0.        ,
        0.        ],
       [0.0138104 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03635043, 0.        , 0.        , ..., 0.02937838, 0.        ,
        0.        ]], dtype=float32)

In [127]:
final_out_2.argmax(dim=1)

tensor([ 89,   1,   1,  92,  89,  92,  34,  92,  92,   1,   1,  92,  34,  92,
         34,  92,  92,   1,   1,  63,  34,  92,  34,  43,  92,   1,   1,  63,
         34,  34,  34,  43,  92,   1,   1,  63,  34,  34, 114,  92,  92,   1,
          1,  63,  34,  34, 114,  92,  92,   1,   1,  63,  34,  34, 114,  92,
         92,   1,   1,  63,  34,  34, 114,  92,  92,   1,   1,  43,  34,  34,
        114,  34,  92,   1,   1,  43,  34,  34, 114,  34,  92,   1,   1,  34,
         34,  34, 114,  34,  43,   1,   1,  34,  34,  34, 114,  34,  43,   1,
          1,  63,  34,  34, 114,  34,  43,   1,   1,  43,  34,  34, 114,  34,
         43,   1,   1,  43,  34,  34, 114,  34,  43,   1,   1,  43,  34,  34,
        114,  34,  43,   1,   1,  43,  34,  34, 114,  77,  92,   1,   1,  43,
         34,  34,  34,  77,  92,   1,   1,  34,  34,  77,  34,  77,  92,   1,
          1,  34,  34,  77, 114,  92,  92,   1,   1,  34,  34,  77,  77,  92,
         92,   1,   1,  34,  34,  43, 114,  89,  92,   1,  43,  

In [102]:
final_out_2.argmax(dim=1).unique(return_counts =True)

(tensor([  0,   1,  34,  43,  63,  77,  89,  92, 114], device='cuda:0'),
 tensor([  2, 167, 214, 128,  34,  86,  29,  93,  47], device='cuda:0'))

In [5]:
all_idx = torch.cat((vp_idx, cp_idx))

In [34]:
all_idx.size()

torch.Size([563])

In [4]:
all_p = np.concatenate((vp, cp))

In [36]:
len(all_idx)

563

tensor([[0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 1., 0.,  ..., 1., 0., 0.]], device='cuda:0')

In [68]:
def convert_to_softmax(out, feat_idx_dict, phone_idx_dict, all_p, all_idx):
    n_phones = len(phone_idx_dict)
    n_phones_full = len(all_idx)
    n = out.size()[0]
    
    vowel_idx = feat_idx_dict["vowel"]
    consonant_idx = feat_idx_dict["consonant"]
    vowels = out[:, vowel_idx]
    consonants = out[:, consonant_idx]
    sums = vowels + consonants
    # There is roughly a 50% chance this phone is non-silence.
    sum_bools = sums > 0.5
    # Take max of vowel vs. consonant
    silence_idx = ~sum_bools
    
    phones = out[sum_bools]
    
    # Output is N x #phones
    final_out = torch.zeros((out.size()[0], n_phones))
    
    all_scores = [phones[:,slice_idx[1:]].sum(dim=1)/(len(slice_idx)-1) for slice_idx in all_p]
    all_scores = torch.stack(all_scores, dim=0)
    
    zero_idx = all_idx == 0
    zero_count = zero_idx.sum()
    if zero_count > 1: # Need to combine multiple 0 outputs
        # Only keep highest zero-score
        max_zero_score = all_scores[zero_idx].max(axis=0)[0]
        # Add 1-dim for concatenation
        max_zero_score.unsqueeze_(0)

        # filter out the original zero rows
        all_scores = torch.cat((max_zero_score, all_scores[~zero_idx,:]), 0).cuda()
        zero = torch.zeros(1)
        all_idx = torch.cat((zero, all_idx[~zero_idx].float())).cuda().long()
    
    lang_scores = torch.zeros((all_scores.shape[1], n_phones))
    for idx, col_idx in enumerate(all_idx.long()):
        lang_scores[:,col_idx] = all_scores[idx,:]
    
    final_out[sum_bools] = lang_scores
    final_out[silence_idx,1] = 1
    # Set zero scores to -inf for softmax
    final_out[final_out == 0] = -math.inf
    final_out = softmax(final_out, dim=1)
#     for row_idx, phone_idx in enumerate(all_idx):
#         final_out[sum_bools][phone_idx] = all_scores[row_idx,:]
    #final_out.masked_scatter()
    
    # Add prediction for silence
    # Get normalised total across all attributes. 1 - total = score for silence 
    #silence_score = 1 - (out.sum(axis=1)/(out.size()[1]))
    #final_out[:,1] = silence_score
    
    return final_out

final_out = convert_to_softmax(out, feat_idx_dict, phone_idx_dict, all_p, all_idx)

In [72]:
final_out.argmax(dim=1)

tensor([63,  1,  1, 63, 34, 92, 34, 63, 63,  1,  1, 63, 34, 63, 34, 63, 63,  1,
         1, 63, 34, 63, 34, 63, 63,  1,  1, 63, 34, 34, 34, 63, 63,  1,  1, 63,
        34, 34, 34, 63, 63,  1,  1, 63, 34, 34, 34, 63, 63,  1,  1, 63, 34, 34,
        34, 63, 63,  1,  1, 63, 34, 34, 34, 34, 63,  1,  1, 63, 34, 34, 34, 34,
        63,  1,  1, 34, 34, 34, 34, 34, 63,  1,  1, 34, 34, 34, 34, 34, 63,  1,
         1, 34, 34, 34, 34, 34, 63,  1,  1, 63, 34, 34, 34, 34, 63,  1,  1, 63,
        34, 34, 34, 34, 63,  1,  1, 63, 34, 34, 34, 34, 63,  1,  1, 63, 34, 34,
        34, 34, 63,  1,  1, 63, 34, 34, 34, 34, 63,  1,  1, 63, 34, 34, 34, 34,
        63,  1,  1, 34, 34, 34, 34, 34, 63,  1,  1, 34, 34, 34, 34, 34, 63,  1,
         1, 34, 34, 34, 34, 34, 63,  1,  1, 34, 34, 43, 34, 34, 63,  1, 63, 34,
        34, 63, 34, 34, 63,  1, 63, 34, 34, 34, 34, 34, 34,  1, 63, 34, 34, 34,
        34, 63, 34,  1, 63, 34, 34, 34, 34, 63, 34,  1,  1, 34, 34, 34, 34, 63,
        34,  1,  1, 63, 34, 34, 34, 63, 

In [51]:
true = mapping_to_labels(mapping, lab_dnn)

In [53]:
np.unique(true, return_counts=True)

(array([  1.,   5.,  14.,  15.,  26.,  34.,  41.,  59.,  60.,  63.,  65.,
         68.,  72.,  75.,  77.,  82.,  89.,  92., 100., 107., 114., 116.]),
 array([165,   6,   3,  48,  17, 136,  10,  23,  16,  70,  46,  49,  21,
         26,  27,   3,  41,   6,  44,   9,  17,  17], dtype=int64))

In [30]:
test[test == 0] = -math.inf

In [32]:
soft2 = softmax(test, dim=1)

In [43]:
preds.unique(return_counts=True)

(tensor([ 0, 34, 43, 63, 77, 92]), tensor([  2, 510,  16, 265,   3,   4]))

In [7]:
final_out.argmax(axis=1)

tensor([63, 34, 34, 63, 34, 92, 34, 63, 63, 34, 34, 63, 34, 63, 34, 63, 63, 34,
        34, 63, 34, 63, 34, 63, 63, 34, 34, 63, 34, 34, 34, 63, 63, 34, 34, 63,
        34, 34, 34, 63, 63, 34, 34, 63, 34, 34, 34, 63, 63, 34, 34, 63, 34, 34,
        34, 63, 63, 34, 34, 63, 34, 34, 34, 34, 63, 34, 34, 63, 34, 34, 34, 34,
        63, 34, 34, 34, 34, 34, 34, 34, 63, 34, 34, 34, 34, 34, 34, 34, 63, 34,
        34, 34, 34, 34, 34, 34, 63, 34, 34, 63, 34, 34, 34, 34, 63, 34, 34, 63,
        34, 34, 34, 34, 63, 34, 34, 63, 34, 34, 34, 34, 63, 34, 34, 63, 34, 34,
        34, 34, 63, 34, 34, 63, 34, 34, 34, 34, 63, 34, 34, 63, 34, 34, 34, 34,
        63, 34, 34, 34, 34, 34, 34, 34, 63, 34, 34, 34, 34, 34, 34, 34, 63, 34,
        63, 34, 34, 34, 34, 34, 63, 34, 63, 34, 34, 43, 34, 34, 63, 34, 63, 34,
        34, 63, 34, 34, 63, 34, 63, 34, 34, 34, 34, 34, 34, 34, 63, 34, 34, 34,
        34, 63, 34, 34, 63, 34, 34, 34, 34, 63, 34, 34, 63, 34, 34, 34, 34, 63,
        34, 34, 63, 63, 34, 34, 34, 63, 

In [10]:
softy = softmax(final_out, dim=1)

In [23]:
final_out[2,:]

tensor([0.0760, 0.0000, 0.0000, 0.0582, 0.0579, 0.0327, 0.0538, 0.0628, 0.0503,
        0.0199, 0.0405, 0.0718, 0.0575, 0.0327, 0.0305, 0.0838, 0.0671, 0.0000,
        0.0300, 0.0790, 0.0319, 0.0284, 0.0118, 0.0260, 0.0609, 0.0291, 0.0377,
        0.0000, 0.0317, 0.0274, 0.0308, 0.0328, 0.0313, 0.0271, 0.0970, 0.0776,
        0.0000, 0.0000, 0.0293, 0.0252, 0.0253, 0.0000, 0.0249, 0.0340, 0.0268,
        0.0291, 0.0293, 0.0252, 0.0273, 0.0326, 0.0281, 0.0000, 0.0789, 0.0632,
        0.0312, 0.0270, 0.0269, 0.0232, 0.0234, 0.0000, 0.0321, 0.0317, 0.0265,
        0.0860, 0.0689, 0.0293, 0.0253, 0.0000, 0.0272, 0.0227, 0.0227, 0.0236,
        0.0324, 0.0286, 0.0268, 0.0332, 0.0277, 0.0371, 0.0319, 0.0297, 0.0328,
        0.0263, 0.0304, 0.0335, 0.0263, 0.0322, 0.0313, 0.0269, 0.0244, 0.0374,
        0.0322, 0.0308, 0.0343, 0.0272, 0.0236, 0.0296, 0.0263, 0.0277, 0.0329,
        0.0284, 0.0399, 0.0319, 0.0309, 0.0327, 0.0273, 0.0267, 0.0000, 0.0323,
        0.0303, 0.0262, 0.0651, 0.0521, 

In [19]:
softy[2,:]

tensor(0.0087, grad_fn=<SelectBackward>)

In [14]:
softy

tensor([[0.0089, 0.0066, 0.0066,  ..., 0.0078, 0.0066, 0.0066],
        [0.0082, 0.0082, 0.0082,  ..., 0.0082, 0.0082, 0.0082],
        [0.0086, 0.0079, 0.0079,  ..., 0.0082, 0.0079, 0.0079],
        ...,
        [0.0098, 0.0066, 0.0066,  ..., 0.0078, 0.0066, 0.0066],
        [0.0089, 0.0068, 0.0068,  ..., 0.0078, 0.0068, 0.0068],
        [0.0093, 0.0070, 0.0070,  ..., 0.0078, 0.0070, 0.0070]],
       grad_fn=<SoftmaxBackward>)

In [52]:
len(phone_idx_dict)

122

In [56]:
final_out = torch.zeros((out.size()[0], 122)).cuda()
dummy = torch.ones((107,800)).cuda()

In [58]:
final_out.scatter(1, all_idx_2.long(), dummy)

RuntimeError: invalid argument 3: Index tensor must be either empty or have same dimensions as input tensor at C:/w/1/s/tmp_conda_3.7_100118/conda/conda-bld/pytorch_1579082551706/work/aten/src\THC/generic/THCTensorScatterGather.cu:115

In [ ]:
    v_scores = [vowels[:,slice_idx[1:]].sum(dim=1)/(len(slice_idx)-1) for slice_idx in vp]
    # Stack to shape [29, N], where each 29 is the score for each vowel
    v_scores = torch.stack(v_scores, dim=0)

In [15]:
phone_nums = [x[0] for x in all_p]

In [20]:
unique = list(set(list(phone_nums)))

In [10]:
final_out.shape

torch.Size([800, 122])

In [8]:
def check_sums(tensor_list, target_sum=None):
    init_shape = tensor_list[0].size()
    if target_sum is None:
        target_sum = init_shape[0]
    sum_true_vals = 0
    for tensor in tensor_list:
        assert tensor.size() == init_shape, "Tensors mismatch in size"
        sum_true_vals += tensor.unique(return_counts=True)[1][1].item()
    return sum_true_vals == target_sum
    
check_sums([vowel_idx, consonant_idx, silence_idx])

True

In [66]:
def split_on_max(atts_list, tensor, feat_idx_dict):
    # Get indices corresponding to articulatory features in question
    slice_idx = [feat_idx_dict[x] for x in atts_list]
    print(slice_idx)
    

atts_list = ["front", "back", "central"]
split_on_max(atts_list, vowels, feat_idx_dict)

[21, 6, 9]


In [130]:
height_list.sort()
height_idx = [feat_idx_dict[x] for x in height_list]
max_idx = test[height_idx].argmax()
height_list[max_idx]

'front'

In [138]:
schwa_idx = [feat_idx_dict[x] for x in ["close-mid", "open-mid", "rounded", "unrounded"]]

In [155]:
sum(test[schwa_idx])

0.5613911182736047

In [168]:
root_dict["front"]

[[0, 'a', 'vowel', 'open', 'front', 'unrounded'],
 [5, 'e', 'vowel', 'close-mid', 'front', 'unrounded'],
 [11, 'i', 'vowel', 'close', 'front', 'unrounded'],
 [43, 'y', 'vowel', 'close', 'front', 'rounded'],
 [52, 'E', 'vowel', 'open-mid', 'front', 'unrounded'],
 [58, 'I', 'vowel', 'near-close', 'front', 'unrounded'],
 [85, 'Y', 'vowel', 'near-close', 'front', 'rounded'],
 [88, '{', 'vowel', 'near-open', 'front', 'unrounded'],
 [91, '2', 'vowel', 'close-mid', 'front', 'rounded'],
 [99, '9', 'vowel', 'open-mid', 'front', 'rounded'],
 [100, '&', 'vowel', 'open', 'front', 'rounded']]

In [166]:
def classify_vowel(phone, height_list, height_idx, openness_list, openness_idx, root_dict):
    # Get most likely height (front/back/central)
    height = height_list[test[height_idx].argmax()]
    if height != "central":
        
    

classify_vowel(test, height_list, height_idx, openness_list, openness_idx, {})

front


In [144]:
(out[:, schwa_idx].sum(dim=1) > 1).unique(return_counts=True)

(tensor([False], device='cuda:0'), tensor([800], device='cuda:0'))

In [171]:
next_step_dict = {}
for height, phones in height_dict.items():
    subsplit = check_phones_split(openness_list, phones)
    subkeys = set()
    for key in subsplit:
        subkeys.add(key)
    next_step_dict[height] = subkeys
next_step_dict

{'front': {'close',
  'close-mid',
  'near-close',
  'near-open',
  'open',
  'open-mid'},
 'back': {'close', 'close-mid', 'near-close', 'open', 'open-mid'},
 'central': {'close', 'close-mid', 'near-close', 'near-open', 'open-mid'}}

In [155]:
def setup(feat_idx_dict):
    openness_list = ["open", "close-mid", "close", "open-mid", "near-close", "near-open"]
    height_list = ["front", "back", "central"]
    height_idx = [feat_idx_dict[x] for x in height_list]
    openness_idx = [feat_idx_dict[x] for x in openness_list]
    return height_list, height_idx, openness_list, openness_idx

def check_phones_split(atts_list, phones, att_vec_idx=None):
    out_split = {}
    for att_idx, att in enumerate(atts_list):
        matching_phones = []
        for phone in phones:
            # Need to do [1:] since first number is phone index itself
            if att in phone[1:]:
                matching_phones.append(phone)
        if len(matching_phones) > 0:
            # Can use the index instead of the name 
            if att_vec_idx is not None:
                out_split[att_vec_idx[att_idx]] = matching_phones
            else:
                out_split[att] = matching_phones
    return out_split


def build_dict_tree(phones, feat_idx_dict, height_list, openness_list):
    root_dict = {} 
    height_idx = [feat_idx_dict[x] for x in height_list]
    openness_idx = [feat_idx_dict[x] for x in openness_list]
    root_dict["operation"] = "max"
    root_dict["indices"] = height_idx
    
    height_dict = check_phones_split(height_idx, phones, att_vec_idx=height_idx)
    print(height_dict)
    

build_dict_tree(vp, feat_idx_dict, height_list, openness_list)

{21: [array([34, 34, 21, 45]), array([52, 12, 21, 45]), array([63, 11, 21, 45]), array([110,  11,  21,  42]), array([15, 35, 21, 45]), array([19, 31, 21, 45]), array([ 0, 31, 21, 42]), array([117,  32,  21,  45]), array([ 4, 12, 21, 42]), array([ 7, 35, 21, 42]), array([ 0, 34, 21, 42])], 6: [array([80, 12,  6, 42]), array([100,  11,   6,  42]), array([11, 34,  6, 45]), array([24, 11,  6, 45]), array([26, 35,  6, 42]), array([ 0, 34,  6, 42]), array([31, 31,  6, 42]), array([ 0, 35,  6, 45]), array([ 6, 12,  6, 45])], 9: [array([ 0, 31,  9, 45]), array([ 0, 31,  9, 42]), array([10, 12, 35,  9]), array([ 0, 11,  9, 42]), array([ 3, 11,  9, 45]), array([ 0, 35,  9, 45]), array([ 0, 35,  9, 42]), array([ 0, 32,  9]), array([ 0, 12,  9, 42])]}


In [172]:
for height, phones in height_dict.items():
    print(height)
    subsplit = check_phones_split(openness_list, phones)
    for sub_phones in subsplit.values():
        print(sub_phones)

front
[[0, 'a', 'vowel', 'open', 'front', 'unrounded'], [100, '&', 'vowel', 'open', 'front', 'rounded']]
[[5, 'e', 'vowel', 'close-mid', 'front', 'unrounded'], [91, '2', 'vowel', 'close-mid', 'front', 'rounded']]
[[11, 'i', 'vowel', 'close', 'front', 'unrounded'], [43, 'y', 'vowel', 'close', 'front', 'rounded']]
[[52, 'E', 'vowel', 'open-mid', 'front', 'unrounded'], [99, '9', 'vowel', 'open-mid', 'front', 'rounded']]
[[58, 'I', 'vowel', 'near-close', 'front', 'unrounded'], [85, 'Y', 'vowel', 'near-close', 'front', 'rounded']]
[[88, '{', 'vowel', 'near-open', 'front', 'unrounded']]
back
[[47, 'A', 'vowel', 'open', 'back', 'unrounded'], [73, 'Q', 'vowel', 'open', 'back', 'rounded']]
[[21, 'o', 'vowel', 'close-mid', 'back', 'rounded'], [97, '7', 'vowel', 'close-mid', 'back', 'unrounded']]
[[37, 'u', 'vowel', 'close', 'back', 'rounded'], [66, 'M', 'vowel', 'close', 'back', 'unrounded']]
[[70, 'O', 'vowel', 'open-mid', 'back', 'rounded'], [81, 'V', 'vowel', 'open-mid', 'back', 'unrounded']]

In [ ]:
vowel_phone_tree = {
    "front": {
        "close": {
            "rounded": {"y"},
            "unrounded": {"i"}            
        },
        "near-close": {
            "rounded": {"Y"},
            "unrounded": {"I"}            
        },
        "near-open": {"{"}, 
        "open": {
            "rounded": {"&"},
            "unrounded": {"a"}
        },
        "open-mid": {
            "rounded": {"9"},
            "unrounded": {"E"}
        }
    },
    "back": {
        "close": {},
        "close-mid": {},
        "near-close:": {},
        "open": {},
        "open-mid": {}
    },
    "central": {
        "close": {},
        "close-mid": {},
        "near-close": {},
        "near-open": {},
        "open-mid": {}
    }
}

In [ ]:
# For splitting the last set of phones
def final_split_rounded(phones):
    n = len(phones)
    # Only one phone, don't need to split
    if n == 1:
        return phones
    elif n == 2:
        

In [137]:
for height, phones in height_dict.items():
    print(height)
    subsplit = check_phones_split(openness_list, phones)
    for sub_phones in subsplit.values():
        print(sub_phones)
        #if len(sub_phones) > 2:
        #    print(sub_phones)

front
[[0, 'a', 'vowel', 'open', 'front', 'unrounded'], [100, '&', 'vowel', 'open', 'front', 'rounded']]
[[5, 'e', 'vowel', 'close-mid', 'front', 'unrounded'], [91, '2', 'vowel', 'close-mid', 'front', 'rounded']]
[[11, 'i', 'vowel', 'close', 'front', 'unrounded'], [43, 'y', 'vowel', 'close', 'front', 'rounded']]
[[52, 'E', 'vowel', 'open-mid', 'front', 'unrounded'], [99, '9', 'vowel', 'open-mid', 'front', 'rounded']]
[[58, 'I', 'vowel', 'near-close', 'front', 'unrounded'], [85, 'Y', 'vowel', 'near-close', 'front', 'rounded']]
[[88, '{', 'vowel', 'near-open', 'front', 'unrounded']]
back
[[47, 'A', 'vowel', 'open', 'back', 'unrounded'], [73, 'Q', 'vowel', 'open', 'back', 'rounded']]
[[21, 'o', 'vowel', 'close-mid', 'back', 'rounded'], [97, '7', 'vowel', 'close-mid', 'back', 'unrounded']]
[[37, 'u', 'vowel', 'close', 'back', 'rounded'], [66, 'M', 'vowel', 'close', 'back', 'unrounded']]
[[70, 'O', 'vowel', 'open-mid', 'back', 'rounded'], [81, 'V', 'vowel', 'open-mid', 'back', 'unrounded']]

In [22]:
def print_set(input_set):
    line = ""
    for attribute in input_set:
        line += attribute + " "
    print(line)

In [10]:
line = ""
for attribute in ca:
    line += attribute + " "
print(line)

velar voiced alveolar glottal lateral coronal dorsal labial-palatal trill stop palatal apical dental plosive approximant bilabial postalveolar pharyngeal fricative affricate voiceless labial alveolo-palatal flap epiglottal labial-velar nasal click palatal-velar uvular retroflex labiodental 


In [60]:
full_dict = {}
for att in place_general:
    att_phones = []
    for phone in cp:
        if att in phone:
            att_phones.append(phone)
    full_dict[att] = att_phones

NameError: name 'place_general' is not defined

In [38]:
place_top = {"coronal", "labial", "dorsal", "pharyngeal", "glottal", "epiglottal", "alveolo-palatal", "labiodental"}
place_coronal = {"dental", "alveolar", "postalveolar", "retroflex", "palatal"}
place_dorsal = {"palatal", "velar", "uvular", "palatal-velar"}
place_labial = {"bilabial", "labiodental", "labial-palatal"}
manner = {"nasal", "stop", "fricative", "approximant", "flap", "trill", "lateral"}

remaining_atts = set()
for att in ca:
    selected_atts = place_top.union(manner)
    selected_atts = selected_atts.union(place_coronal)
    selected_atts = selected_atts.union(place_dorsal)
    selected_atts = selected_atts.union(place_labial)
    if att not in selected_atts:
        remaining_atts.add(att)

print_set(remaining_atts)

voiced labial-velar affricate apical click voiceless plosive 


In [101]:

class BinaryTree(object):
    def __init__(self, key):
        self.left = None
        self.right = None
        self.key = key
        self.split_idx = None
        self.left_is_leaf = False
        self.right_is_leaf = False
        
    def display(self):
        lines, _, _, _ = self._display_aux()
        for line in lines:
            print(line)
            
    def save(self, filepath):
        lines, _, _, _ = self._display_aux()
        with open(filepath, "w") as f:
            for line in lines:
                f.write(line + "\n")

    def _display_aux(self):
        """Returns list of strings, width, height, and horizontal coordinate of the root."""
        # No child.
        if self.right is None and self.left is None:
            line = '%s' % self.key
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        # Only left child.
        if self.right is None:
            lines, n, p, x = self.left._display_aux()
            s = '%s' % self.key
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s
            second_line = x * ' ' + '/' + (n - x - 1 + u) * ' '
            shifted_lines = [line + u * ' ' for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

        # Only right child.
        if self.left is None:
            lines, n, p, x = self.right._display_aux()
            s = '%s' % self.key
            u = len(s)
            first_line = s + x * '_' + (n - x) * ' '
            second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
            shifted_lines = [u * ' ' + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        # Two children.
        left, n, p, x = self.left._display_aux()
        right, m, q, y = self.right._display_aux()
        s = '%s' % self.key
        u = len(s)
        first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
        second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
        if p < q:
            left += [n * ' '] * (q - p)
        elif q < p:
            right += [m * ' '] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2

In [104]:
# Check score
def score_split(feature, phones):
    n_tr = 0
    n_fa = 0
    for phone in phones:
        if feature in phone[2:]:
            n_tr += 1
        else:
            n_fa += 1
    total = n_tr + n_fa
    percent = n_tr / total
    return abs(percent - 0.5)

def best_split(phones, allowable_atts):
    atts_list = []
    atts_scores = []
#     print("phones: {}".format(str(phones)))
#     print("all atts: {}".format(str(all_attributes)))
#     print("selected atts: {}".format(str(selected_atts)))
    for attribute in allowable_atts:
        score = score_split(attribute, phones)
        atts_list.append(attribute)
        atts_scores.append(score)
    best_idx = np.argmin(atts_scores)
    return atts_list[best_idx]

def do_split(feature, phones, allowable_atts):
    true_phones = []
    false_phones = []
    true_attributes = set()
    false_attributes = set()
    for phone in phones:
        if feature in phone:
            true_phones.append(phone)
            for attribute in phone[2:]:
                if attribute in allowable_atts:
                    true_attributes.add(attribute)
        else:
            false_phones.append(phone)
            for attribute in phone[2:]:
                if attribute in allowable_atts:
                    false_attributes.add(attribute)
    return true_phones, false_phones, true_attributes, false_attributes    
    


# True values (> 0.5 are on the left, false values on the right)
def build_tree(phones, allowable_atts, feat_idx_dict, use_indices=True, is_first=False):

    # Find best feature to split on. Update binary tree
    best_feat = best_split(phones, allowable_atts)
    
    if not use_indices:
        curr_tree = BinaryTree(best_feat)
    else:
        curr_tree = BinaryTree(feat_idx_dict[best_feat])
    allowable_atts.remove(best_feat)
    tr_phones, fa_phones, tr_atts, fa_atts = do_split(best_feat, phones, allowable_atts)
    # Update leaves of tree if possible
    if len(tr_phones) == 1:
        # Convert e.g. ['b consonant voiced bilabial stop labial'] to "b"
        if not use_indices:
            tr_phone = tr_phones[0][1]
        else:
            tr_phone = tr_phones[0][0]
        curr_tree.left = BinaryTree(tr_phone)
    else:
        curr_tree.left = build_tree(tr_phones, tr_atts, feat_idx_dict, use_indices)
        
        
    if len(fa_phones) == 1:
        if not use_indices:
            fa_phone = fa_phones[0][1]
        else:
            fa_phone = fa_phones[0][0]
        curr_tree.right = BinaryTree(fa_phone)
    else:
        curr_tree.right = build_tree(fa_phones, fa_atts, feat_idx_dict, use_indices)
    return curr_tree
    
curr_tree = build_tree(cp, ca, feat_idx_dict, use_indices=False, is_first=True)
curr_tree.save("consonant_tree")

vowel_tree = build_tree(vp, va, feat_idx_dict, use_indices=False, is_first=True)
vowel_tree.save("vowel_tree")

NameError: name 'BinaryTree' is not defined

In [ ]:
out_replicated = torch.repeat_interleave(out, repeats=num_phones, dim=0)
mapping_t = torch.from_numpy(mapping).float().to(device)
mapping_t = mapping_t.repeat(800, 1)
dists = F.pairwise_distance(out_replicated, mapping_t)
cosines = F.cosine_similarity(out_replicated, mapping_t)
cosines = cosines.view(n, num_phones)
dists = dists.view(n, num_phones)